In [ ]:
import cv2
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity

detector = MTCNN()
embedder = FaceNet()

def get_embedding(face_img):
    face_img = cv2.resize(face_img, (160, 160))
    face_img = face_img.astype("float32")
    face_img = np.expand_dims(face_img, axis=0)
    embedding = embedder.embeddings(face_img)
    return embedding[0]

ref_img = cv2.imread("data/reference.jpg")
ref_rgb = cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB)

ref_faces = detector.detect_faces(ref_rgb)
x, y, w, h = ref_faces[0]["box"]
ref_face = ref_rgb[y:y+h, x:x+w]

ref_embedding = get_embedding(ref_face)

cap = cv2.VideoCapture(0)

THRESHOLD = 0.7

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)

    for face in faces:
        x, y, w, h = face["box"]
        face_img = rgb[y:y+h, x:x+w]

        emb = get_embedding(face_img)
        sim = cosine_similarity([ref_embedding], [emb])[0][0]

        if sim > THRESHOLD:
            label = f"Matched ({sim:.2f})"
            color = (0, 255, 0)
        else:
            label = f"Unknown ({sim:.2f})"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()